In [1]:
import bs4 as bs, urllib, pandas as pd, numpy as np

Parse past X years

In [2]:
keyword='medve'
baseurl=u'https://szekelyhon.ro/kereses?op=search&src_words='

In [30]:
start='2019-09'
end='2019-11'
dates=[]
datelist = pd.date_range(start=pd.to_datetime(start), end=pd.to_datetime(end), freq='M').tolist()
for date in datelist:
    dates.append(str(date)[:10])

In [31]:
dates[:5]

['2019-09-30', '2019-10-31']

In [32]:
def extractor(time1,time2):
    time1=dates[i]
    time2=dates[i+1]
    print('Parsing...',time1,'-',time2)
    url=baseurl+keyword+'&src_time1='+time1+'&src_time2='+time2
    html = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(html,'lxml')
    return soup.findAll("div", {"class": "cikkocka2c"})

In [33]:
divs=[]
for i in range(len(dates)-1):
    time1=dates[i]
    time2=dates[i+1]
    divs.append(extractor(time1,time2))

Parsing... 2019-09-30 - 2019-10-31


In [34]:
def date_hu_en(i):
    date=i[6:-4]
    if date=='augusztus': m='08'
    elif date=='december': m='12'
    elif date=='február': m='02'
    elif date=='január': m='01'
    elif date=='július': m='07'
    elif date=='június': m='06'
    elif date=='május': m='05'
    elif date=='március': m='03'
    elif date=='november': m='11'
    elif date==u'október': m='10'
    elif date==u'szeptember': m='09'
    elif date==u'április': m='04'
    else: return date
    return i[:4]+'-'+m+'-'+i[-3:-1]

In [35]:
def find_all(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

In [36]:
from utils import text_processor

In [37]:
hirek=[]
tagset=set()
for i in range(len(dates)-1):
    time2=dates[i+1]
    divgroup=divs[i]
    for div in divgroup:
        icat=''
        img=div.find('img')
        if img !=None: 
            img=img['src']
            #infer image category from image link
            icats=find_all(img,'/')
            if len(icats)>4:
                icat=img[icats[3]+1:icats[4]]
        tags=div.find("div", {"class": "tags_con1"})
        if tags!=None: 
            tags=[j.text.strip() for j in tags.findAll('div')]
        idiv=div.find("div", {"class": "catinner"})
        if idiv!=None:
            idiv=idiv.find('div')
            content=div.find('p')
            date=idiv.text[idiv.text.find('20'):idiv.text.find(',')]
            title=div.find('h2').text
            if content==None:
                sdiv=str(div)[::-1]
                content=sdiv[:sdiv.find('>a/<')].replace('\r','').replace('\t','').replace('\n','')[::-1][:-6]
            else: content=content.text
            content=content.replace('</div><div class="clear"></div></div><div class="clear"></div>','')
            link=div.findAll('a')[-1]['href']
            #infer category from link
            cats=find_all(link,'/')
            if len(cats)>3:
                cat=link[cats[2]+1:cats[3]]
            else: cat=''
            #infer attack from plain text
            relevant,severity,deaths=text_processor(title,content)
            if tags!=None:
                notags=[u'Húsvét',u'Film',u'Egészségügy',u'Külföld',u'Színház',u'Ünnep']
                for notag in notags:
                    if notag in tags:
                        relevant=-1
                        break
            if ((relevant>-1)&\
                (cat not in ['sport','muvelodes','sms-e-mail-velemeny','tusvanyos'])&\
                (title not in [u'Röviden'])):
                if tags!=None: 
                    tagset=tagset.union(set(tags))
                    if 'medve' in tags:
                        relevant=1
                hirek.append({'date':date_hu_en(date),
                              'hudate':date,
                              'title':title,
                              'image':img,
                              'tags':repr(tags),
                              'content':content,
                              'link':link,
                              'category':cat,
                              'icategory':icat,
                              'relevant':relevant,
                              'severity':severity,
                              'deaths':deaths,
                              'duplicate':0
                               })

Összes medvés hír

In [38]:
df=pd.DataFrame().from_dict(hirek)
df['date']=pd.to_datetime(df['date'])
df=df.sort_values('date').drop_duplicates().reset_index(drop=True)

In [39]:
len(hirek)

13

Save to medve Excel. Manual curation

In [40]:
dm=df[[ 'date', 'hudate', 'link','image', 'category','icategory','tags','title',
       'content']]
dc=df[['title','content','relevant', 'severity','deaths','duplicate']]

In [41]:
#save parsed data
dm.to_excel('data/szekelyhon_medve.xlsx')

In [42]:
#save data for curation
#1 if you dont have savedata yet
existing_savedata=False
if not existing_savedata:
    dc.to_excel('data/szekelyhon_medve_curated.xlsx')
#2 if you already have savedata
else:
    dc2=pd.read_excel('data/szekelyhon_medve_curated.xlsx')
    dc2.combine_first(dc).to_excel('data/szekelyhon_medve_curated.xlsx')

Open `data/szekelyhon_medve_curated` and manually confirm each case.

Relevant = Medves cikk-e vagy sem: 1-igen, 0-nem biztos, -1:biztosan nem   
Deaths = Halalok szama (ha ismert)  
Severity = Sulyossag: 0-mas jellegu hir, 1-latas, 2-allat-tamadas, 3-ember-tamadas  
Duplicate = 0: Eredeti cikk, 1: Masolat, 2: Osszegzes